In [1]:
import pandas as pd
import tarfile
import zipfile
import gzip
import io
import csv
from data.preprocessing import SOHO, interval_years

In [2]:
scrap_date = interval_years('20200901', '20221010')
satellite = SOHO()
monitor = satellite.CELIAS_Proton_Monitor_data(scrap_date)

100%|██████████| 37024528/37024528 [00:25<00:00, 1465235.22it/s]


In [4]:
df = monitor

In [5]:
df

,YY,MON,DY,DOY:HH:MM:SS,SPEED,Np,Vth,N/S,V_He
0,22,Jan,1,001:01:46:58,474,2.56,37,-4.5,489
1,22,Jan,1,001:01:52:00,458,1.83,33,-2.4,471
2,22,Jan,1,001:01:57:03,456,1.93,33,-3.4,468
3,22,Jan,1,001:02:02:03,457,1.93,32,-4.4,468
4,22,Jan,1,001:02:07:06,459,1.86,38,-2.6,476
...,...,...,...,...,...,...,...,...,...
275115,20,Dec,31,366:23:35:08,404,2.53,28,-0.1,413
275116,20,Dec,31,366:23:40:12,397,2.68,26,-0.3,404
275117,20,Dec,31,366:23:45:13,405,1.95,31,-0.7,416
275118,20,Dec,31,366:23:50:15,398,2.28,30,-0.6,408


In [13]:
df['DOY:HH:MM:SS'][0].split(':')

['001', '01', '46', '58']

In [15]:

# Split the "DOY:HH:MM:SS" column into separate columns
df[['DOY', 'HH', 'MM', 'SS']] = df['DOY:HH:MM:SS'].str.split(':', expand=True)

# Create a 'datetime' column by combining the separate columns
df['datetime'] = pd.to_datetime(
    '20' + df['YY'].apply(str) + df['MON'] + df['DY'].apply(str) + ' ' +
    df['DOY'] + ' ' + df['HH'] + ':' +
    df['MM'] + ':' + df['SS'],
    format='%Y%b%d %j %H:%M:%S')

In [22]:
df.set_index('datetime', axis = 1).drop(['YY', 'MON', 'DY', 'DOY', 'DOY:HH:MM:SS', 'HH', 'MM', 'SS', 'datetime'], axis =1)

TypeError: DataFrame.set_index() got an unexpected keyword argument 'axis'

ftp://sohoftp.nascom.nasa.gov/qkl/lasco/quicklook/level_05/


In [23]:
from sunpy.net import Fido, attrs as a
import astropy.units as u
import cdflib
#copy for COSTEP
result = Fido.search(a.Time('1995/1/1', '2023/12/31'), a.Instrument.costep, a.Sample(5*u.minute))
downloaded_files = Fido.fetch(result, path=f'./data/SOHO_L2/COSTEP_EPHIN_5min/data') 
cdf_file = cdflib.CDF('./data/SOHO_L2/COSTEP_EPHIN_5min/data/costep_ephin_l3_l31_1440_60_30_10_5_1_min_cdf_tar')
variables = cdf_file.cdf_info()
for var in variables['zVariables']:
    print(var)

Files Downloaded: 100%|██████████| 1/1 [01:06<00:00, 66.87s/file]


OSError: /home/jenci/Desktop/SMFGF-SpaceApps/data/SOHO_L2/COSTEP_EPHIN_5min/data/costep_ephin_l3_l31_1440_60_30_10_5_1_min_cdf_tar is not a CDF file or a non-supported CDF!

In [24]:
import sunpy
sunpy.io.read_file('./data/SOHO_L2/COSTEP_EPHIN_5min/data/costep_ephin_l3_l31_1440_60_30_10_5_1_min_cdf_tar')

AttributeError: module 'sunpy' has no attribute 'io'

In [22]:
downloaded_files

['data/SOHO_L2/COSTEP_EPHIN_5min/data/costep_ephin_l3_l31_1440_60_30_10_5_1_min_cdf_tar.gz']

In [13]:
print(a.Instrument)

sunpy.net.attrs.Instrument

Specifies the Instrument name for the search.

       Attribute Name       ...
--------------------------- ...
aia                         ...
bcs                         ...
be_continuum                ...
be_halpha                   ...
bigbear                     ...
caii                        ...
cds                         ...
celias                      ...
cerrotololo                 ...
chp                         ...
cook                        ...
costep                      ...
cp                          ...
dpm                         ...
eis                         ...
eit                         ...
elteide                     ...
erne                        ...
eui                         ...
eve                         ...
eve                         ...
film                        ...
five_12_channelmagnetograph ...
foxsi                       ...
gbm                         ...
goes                        ...
golf                        .

In [20]:
#COPY FOR SDO/AIA
result = Fido.search(a.Time('1995/1/1', '2023/12/31'), a.Instrument.aia, a.Wavelength(193*u.AA))
result

LINE 1: SELECT count(*) FROM vso.aia_lev1 WHERE 1=1 AND  (wave BETWE...
                             ^
 [sunpy.net.vso.vso]
LINE 1: SELECT count(*) FROM vso.aia_lev1 WHERE 1=1 AND  (wave BETWE...
                             ^



Start Time,End Time,Source,Instrument,Wavelength,Provider,Physobs,Wavetype,Extent Type,Size
,,,,Angstrom,,,,,Mibyte
Time,Time,str3,str3,float64[2],str8,str9,str6,str8,float64
2023-11-29 18:54:00.000,2023-11-29 18:54:00.000,SDO,AIA,193.0 .. 193.0,SDAC_AIA,intensity,NARROW,FULLDISK,4.01562
2023-11-29 18:57:00.000,2023-11-29 18:57:00.000,SDO,AIA,193.0 .. 193.0,SDAC_AIA,intensity,NARROW,FULLDISK,4.01562
2023-11-29 19:00:00.000,2023-11-29 19:00:00.000,SDO,AIA,193.0 .. 193.0,SDAC_AIA,intensity,NARROW,FULLDISK,4.01562
2023-11-29 19:03:00.000,2023-11-29 19:03:00.000,SDO,AIA,193.0 .. 193.0,SDAC_AIA,intensity,NARROW,FULLDISK,4.01562
2023-11-29 19:06:00.000,2023-11-29 19:06:00.000,SDO,AIA,193.0 .. 193.0,SDAC_AIA,intensity,NARROW,FULLDISK,4.01562
2023-11-29 19:09:00.000,2023-11-29 19:09:00.000,SDO,AIA,193.0 .. 193.0,SDAC_AIA,intensity,NARROW,FULLDISK,4.01562
2023-11-29 19:12:00.000,2023-11-29 19:12:00.000,SDO,AIA,193.0 .. 193.0,SDAC_AIA,intensity,NARROW,FULLDISK,4.01562
2023-11-29 19:15:00.000,2023-11-29 19:15:00.000,SDO,AIA,193.0 .. 193.0,SDAC_AIA,intensity,NARROW,FULLDISK,4.01562


In [ ]:
result = Fido.search(a.Time('1995/1/1', '2023/12/31'), a.Instrument.hmi, a.Sample(5*u.minute))
result

In [3]:
from sunpy.net import Fido, attrs as a
a.jsoc.Series.aia_lev1

<sunpy.net.jsoc.attrs.Series(aia.lev1: AIA Level 1) object at 0x7f4926282a40>

In [6]:
res = Fido.search(a.Time('2000-01-01T00:00:00', '2023-01-01T01:00:00'),

                  a.jsoc.Series('hmi.v_45s')) 
res